In [6]:
# -*- coding: utf-8 -*-
import os
import inspect
import numpy as np
from pycode import reader,plt,process,FileChoice,testdata
import plotly
from scipy import interpolate
from scipy import signal
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.io.wavfile import read as readwav

def Csv(path):
    df = reader.CsvReader(path)
    return df["X"],df["Y"],df["S"]

def Wave(path):
    df = readwav(path)
    audio = df[1]
    return audio

def ArgName(func):
    arg_names = func.__code__.co_varnames[:func.__code__.co_argcount]
    return arg_names

def F_Fig(data=None,layout=None,frames=None):
    figure = go.Figure(data,layout,frames)
    return figure


def F_Layout(xran=None,yran=None,title=None,buttons=False):
    layout=go.Layout(
        xaxis=dict(range=xran),
        yaxis=dict(range=yran),
        title=title,
    )
    return layout

def F_3DLayout(xran=None,yran=None,zran=None,title=None):
    layout=go.Layout(
        scene = dict(
            xaxis=dict(range=xran,title="freq"),
            yaxis=dict(range=yran,title="F.real"),
            zaxis=dict(range=zran,title="F.iamg"),
        ),
    )
    return layout

def F_AnimLayout(frames,xran=None,yran=None,zran=None,title=None):
    def frame_args(duration):
        return {"frame": {"duration": duration},"mode": "immediate","fromcurrent": True,"transition": {"duration": duration, "easing": "linear"},}

    layout=go.Layout(
    ##
        xaxis=dict(range=[-30,30]),
        yaxis=dict(range=[-30,30]),
    ##
        title=title,
        updatemenus = [{
            "buttons": [{"args": [None, frame_args(50)],
                    "label": "&#9654;", # play symbol
                    "method": "animate",
                },
                {
                    "args": [[None], frame_args(0)],
                    "label": "&#9724;", # pause symbol
                    "method": "animate",
                },
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 70},
            "type": "buttons",
            "x": 0.1,
            "y": 0,
        }],
        sliders = [{
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(frames)
                ],
            }
        ]
    )
    return layout

def F_Anim3DLayout(frames,xran=None,yran=None,zran=None,title=None):
    def frame_args(duration):
        return {"frame": {"duration": duration},"mode": "immediate","fromcurrent": True,"transition": {"duration": duration, "easing": "linear"},}
    layout=go.Layout(
        title=title,
        scene=dict(
            xaxis=dict(range=xran),
            yaxis=dict(range=yran),
            zaxis=dict(range=zran),
            aspectratio=dict(x=1, y=1, z=1),
        ),
        updatemenus = [
        {
            "buttons": [
                {
                    "args": [None, frame_args(50)],
                    "label": "&#9654;", # play symbol
                    "method": "animate",
                },
                {
                    "args": [[None], frame_args(0)],
                    "label": "&#9724;", # pause symbol
                    "method": "animate",
                },
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 70},
            "type": "buttons",
            "x": 0.1,
            "y": 0,
        }],
        sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(frames)
                ],
            }
        ]
    )
    return layout

def F_Line(x,y,color=None):
    fig = go.Scatter(x=x,y=y,marker_color=color)
    return fig

def F_3DLine(x,y,z,color=None):
    fig =go.Scatter3d(
            x=x,
            y=y,
            z=z,
            showlegend=False,
            mode="lines",
            line=dict(color=color,width=2),
            )
    return fig

def F_2DHist(x,y):
    fig =go.Histogram2dContour(
            x=x,
            y=y,
            colorscale="inferno",
            showscale=False,
            reversescale=True
            )
    return fig

def F_Hist(x=None,y=None,color=None):
    fig =go.Histogram(
            x=x,
            y=y,
            marker_color=color
            )
    return fig

def F_Polar(r,theta,color):
    fig = go.Scatterpolar(
            r = r,
            theta = theta,
            mode="lines",
            line=dict(color=color,width=2),
            )
    return fig

##e.g. [[figures],[rows],[cols]] = [[figA, figB, figC],[1, 1, 1],[1, 2, 3]]
def SubFig(figs,layout,color=None,xran=None,yran=None,title=None):
    maxrows = max(figs[1])
    mincols = max(figs[2])
    fig = make_subplots(rows=maxrows,cols=maxcols)
    fig.add_trace(figs)
    layout=go.Layout(
            xaxis=dict(range=xran),
            yaxis=dict(range=yran),
            title=title,
        )
    fig.update_layout(layout)
    return fig

def Figu(x,y,color=None,xran=None,yran=None,title=None):
    fig=go.Figure(
        data=go.Scatter(x=x,y=x,marker_color=color),
        layout=go.Layout(
            xaxis=dict(range=xran),
            yaxis=dict(range=yran),
            title=title,
        )
    )
    return fig

def Figu2(x,y,xran=None,yran=None,title=None):
    fig=go.Figure(
        data=F_2DHist(x,y),
        layout=go.Layout(
            xaxis=dict(range=xran),
            yaxis=dict(range=yran),
            title=title,
        )
    )
    return fig
    
def F_Frame(data):
    frame = [go.Frame(data=fig_data) for fig_data in data]
    return frame
    
def AnimFig(datasets,layoutsets=None):
    frames = F_Frame(data=[_ for _ in datasets])
    layout = F_AnimLayout(frames)
    fig = F_Fig(data=datasets[0],layout=layout,frames=frames)
    return fig

def Pngprint(fig,path):
    fig.write_image(path)

def HtmlExport(fig,path):
    fig.write_html(path)
    
def sep(df):
    separate=[[df[i:i+1000]] for i in range(0,6000,1000)]
    return separate

#!!!#!!!#!!!#
def Roller(df,shift=100,num=1000):
    rol = [[np.roll(df,shift*(-i))[:num]] for i in range(int(len(df)/shift)-int(num/shift)+1)]
    return rol
#!!!#!!!#!!!#

def Liner(n,t,x):
    l=interpolate.interp1d(t,x,fill_value='extrapolate')
    lx=l(n)
    return n,lx

def Hanning(n):
    hann=signal.hann(n)
    return hann

def Rectangle(n):
    rec=signal.boxcar(n)
    return rec

def RecWin(low,high,end):
    recta = np.append((Rectangle(low)-1),Rectangle(high))
    recta = np.append(recta,(Rectangle(end-high)-1))
    return recta


    
def T_Sig2amp(time,sig_x,sig_y,Sigcolor_x,Sigcolor_y,Ampcolor_x,Ampcolor_y,SigRan_x,SigRan_y,AmpRan_x,AmpRan_y,outputpath):
    #init
    dt=0.01
    a,b=time[0],np.ceil(time[-1])

    #signal
    sig_x -= np.mean(sig_x)
    sig_y -= np.mean(sig_y)

    #fourir
    freq,F_x_abs_amp,F_x = process.Sig2amp(time,sig_x,dt)
    freq,F_y_abs_amp,F_y = process.Sig2amp(time,sig_y,dt)
    freq_half = int(max(freq)/2)

    #peak
    peak_x = max(F_x_abs_amp[1:])
    peak_index_x = np.argmax(F_x_abs_amp[1:])+1
    peak_freq_x = freq[peak_index_x]

    peak_y = max(F_y_abs_amp[1:])
    peak_index_y = np.argmax(F_y_abs_amp[1:])+1
    peak_freq_y = freq[peak_index_y]

    #polar
    sig_r,sig_phi=process.cart2pol(sig_x,sig_y)

    #invers
    inv_sig_x=process.Amp2sig(F_x)
    inv_sig_y=process.Amp2sig(F_y)

    ## Visualize ##
    #3D
    sig_3d_data = F_3DLine(x=freq,y=F_x.real,z=F_x.imag,color=Ampcolor_x)
    sig_3d_layout = F_3DLayout(title="F.real & F.imag {} to {}".format(a,b))
    sig_3d_fig = F_Fig(sig_3d_data,sig_3d_layout)
    sig_3d_path = outputpath+r"\F.real_&_F.imag {} to {}.html".format(a,b)
    HtmlExport(sig_3d_fig,sig_3d_path)

    #polar
    #HtmlExport((F_Fig((F_Polar(r=sig_r,theta=sig_phi,color=Ampcolor_x)),(F_Layout2(title="pol x & pol y {} to {}".format(a,b))))),outputpath+r"\polar sig {} to {}.html".format(a,b))

    #Signal
    sig_x_data = F_Line(x=time,y=sig_x,color=Sigcolor_x)
    sig_x_layout = F_Layout(yran=SigRan_x,title="Signal X {} to {}.png".format(a,b))
    sig_x_fig = F_Fig(sig_x_data,sig_x_layout)
    sig_x_path = outputpath+r"\Signal_X {} to {}.png".format(a,b)
    Pngprint(sig_x_fig,sig_x_path)
    
    sig_y_data = F_Line(x=time,y=sig_y,color=Sigcolor_y)
    sig_y_layout = F_Layout(yran=SigRan_y,title="Signal Y {} to {}.png".format(a,b))
    sig_y_fig = F_Fig(sig_y_data,sig_y_layout)
    sig_y_path = outputpath+r"\Signal_y {} to {}.png".format(a,b)
    Pngprint(sig_y_fig,sig_y_path)
    
    #Amplitude
    amp_x_data = F_Line(x=freq,y=F_x_abs_amp,color=Ampcolor_x)
    amp_x_layout = F_Layout(xran=[0,freq_half],yran=AmpRan_x,title="Amplitude_X {} to {}".format(a,b) )
    amp_x_fig = F_Fig(amp_x_data,amp_x_layout)
    amp_x_path = outputpath+r"\Amplitude_X {} to {}.png".format(a,b)
    Pngprint(amp_x_fig,amp_x_path)

    amp_y_data = F_Line(x=freq,y=F_y_abs_amp,color=Ampcolor_y)
    amp_y_layout = F_Layout(xran=[0,freq_half],yran=AmpRan_y,title="Amplitude_Y {} to {}".format(a,b) )
    amp_y_fig = F_Fig(amp_y_data,amp_y_layout)
    amp_y_path = outputpath+r"\Amplitude_Y {} to {}.png".format(a,b)
    Pngprint(amp_y_fig,amp_y_path)
    
    #HeatMap
    heat_data = F_2DHist(sig_x,sig_y)
    heat_layout = F_Layout(xran=SigRan_x,yran=SigRan_y,title="HeatMap {} to {}".format(a,b))
    heat_fig = F_Fig(heat_data,heat_layout)
    heat_path = outputpath+r"\HeatMap {} to {}.png".format(a,b)
    Pngprint(heat_fig,heat_path)
    

    #Inv
    inv_sig_x_data = F_Line(x=time,y=inv_sig_x,color=Sigcolor_x)
    inv_sig_x_layout = F_Layout(yran=SigRan_x,title="inv_Signal X {} to {}.png".format(a,b))
    inv_sig_x_fig = F_Fig(inv_sig_x_data,inv_sig_x_layout)
    inv_sig_x_path = outputpath+r"\inv_Signal_X {} to {}.png".format(a,b)
    Pngprint(inv_sig_x_fig,inv_sig_x_path)
    
    inv_sig_y_path = outputpath+r"\inv_Signal_y {} to {}.png".format(a,b)
    inv_sig_y_data = F_Line(x=time,y=inv_sig_y,color=Sigcolor_y)
    inv_sig_y_layout = F_Layout(yran=SigRan_y,title="inv_Signal Y {} to {}.png".format(a,b))
    inv_sig_y_fig = F_Fig(inv_sig_y_data,inv_sig_y_layout)
    Pngprint(inv_sig_y_fig,inv_sig_y_path)

    #noise_rejection_Amp(cut=10freq_over)
    cut_freq = 50
    nr_F_x = np.copy(F_x)
    nr_F_y = np.copy(F_y)
    nr_F_x[(freq>cut_freq)] = 0
    nr_F_y[(freq>cut_freq)] = 0
    nr_F_x_abs_amp = np.abs(nr_F_x)/len(freq)*2
    nr_F_y_abs_amp = np.abs(nr_F_y)/len(freq)*2
    nr_F_x_abs_amp[0] = nr_F_x_abs_amp[0]/2
    nr_F_y_abs_amp[0] = nr_F_y_abs_amp[0]/2

    nr_F_x_abs_amp_data = F_Line(x=freq,y=nr_F_x_abs_amp,color=Ampcolor_x)
    nr_F_x_abs_amp_layout = F_Layout(xran=[0,freq_half],yran=AmpRan_x,title="nr_Amplitude_X {} to {}".format(a,b))
    nr_F_x_abs_amp_fig = F_Fig(nr_F_x_abs_amp_data,nr_F_x_abs_amp_layout)
    nr_F_x_abs_amp_path = outputpath+r"\nr_Amplitude_X {} to {}.png".format(a,b)
    Pngprint(nr_F_x_abs_amp_fig,nr_F_x_abs_amp_path)
    
    nr_F_y_abs_amp_data = F_Line(x=freq,y=nr_F_y_abs_amp,color=Ampcolor_y)
    nr_F_y_abs_amp_layout = F_Layout(xran=[0,freq_half],yran=AmpRan_y,title="nr_Amplitude_Y {} to {}".format(a,b))
    nr_F_y_abs_amp_fig = F_Fig(nr_F_y_abs_amp_data,nr_F_y_abs_amp_layout)
    nr_F_y_abs_amp_path = outputpath+r"\nr_Amplitude_Y {} to {}.png".format(a,b)
    Pngprint(nr_F_y_abs_amp_fig,nr_F_y_abs_amp_path)

    #noise_rejection_invSig(cut=10freq_over)
    nr_i_sig_x=process.Amp2sig(nr_F_x)
    nr_i_sig_y=process.Amp2sig(nr_F_y)
    nr_i_sig_x*=2
    nr_i_sig_y*=2

    nr_i_sig_x_data = [F_Line(x=time,y=sig_x,color=Sigcolor_x),F_Line(x=time,y=nr_i_sig_x,color=Ampcolor_y)]
    nr_i_sig_x_layout = F_Layout(yran=SigRan_x,title="nr_invSignal of X {} to {}".format(a,b))
    nr_i_sig_x_fig = F_Fig(nr_i_sig_x_data,nr_i_sig_x_layout)
    nr_i_sig_x_path = outputpath+r"\nr_invSignal_X {} to {}.png".format(a,b)
    Pngprint(nr_i_sig_x_fig,nr_i_sig_x_path)

    nr_i_sig_y_data = [F_Line(x=time,y=sig_y,color=Sigcolor_y),F_Line(x=time,y=nr_i_sig_y,color=Ampcolor_x)]
    nr_i_sig_y_layout = F_Layout(yran=SigRan_y,title="nr_invSignal of Y {} to {}".format(a,b))
    nr_i_sig_y_fig = F_Fig(nr_i_sig_y_data,nr_i_sig_y_layout)
    nr_i_sig_y_path = outputpath+r"\nr_invSignal_Y {} to {}.png".format(a,b)
    Pngprint(nr_i_sig_y_fig,nr_i_sig_y_path)

    #noise_rejection_HeatMap
    nr_i_heat_data = F_2DHist(nr_i_sig_x,nr_i_sig_y)
    nr_i_heat_layout = F_Layout(xran=SigRan_x,yran=SigRan_y,title="nr_i_HeatMap {} to {}".format(a,b))
    nr_i_heat_fig = F_Fig(nr_i_heat_data,nr_i_heat_layout)
    nr_i_heat_path = outputpath+r"\nr_i_HeatMap {} to {}.png".format(a,b)
    Pngprint(nr_i_heat_fig,nr_i_heat_path)

    #peak_Amp(uncut=just peak)
    p_F_x = np.copy(F_x)
    p_F_y = np.copy(F_y)
    p_F_x[(freq!=peak_freq_x)] = 0
    p_F_y[(freq!=peak_freq_y)] = 0
    p_F_x_abs_amp = np.abs(p_F_x)/len(freq)*2
    p_F_y_abs_amp = np.abs(p_F_y)/len(freq)*2
    p_F_x_abs_amp[0] = p_F_x_abs_amp[0]/2
    p_F_y_abs_amp[0] = p_F_y_abs_amp[0]/2

    p_F_x_abs_amp_data = [F_Line(x=freq,y=F_x_abs_amp,color=Ampcolor_x),F_Line(x=freq,y=p_F_x_abs_amp,color=Sigcolor_x)]
    p_F_x_abs_amp_layout = F_Layout(xran=[0,freq_half],yran=AmpRan_x,title="peak_Amplitude_X {} to {}".format(a,b))
    p_F_x_abs_amp_fig = F_Fig(p_F_x_abs_amp_data,p_F_x_abs_amp_layout)
    p_F_x_abs_amp_path = outputpath+r"\p_Amplitude_X {} to {}.png".format(a,b)
    Pngprint(p_F_x_abs_amp_fig,p_F_x_abs_amp_path)

    p_F_y_abs_amp_data = [F_Line(x=freq,y=F_y_abs_amp,color=Ampcolor_y),F_Line(x=freq,y=p_F_y_abs_amp,color=Sigcolor_y)]
    p_F_y_abs_amp_layout = F_Layout(xran=[0,freq_half],yran=AmpRan_y,title="peak_Amplitude_Y {} to {}".format(a,b))
    p_F_y_abs_amp_fig = F_Fig(p_F_y_abs_amp_data,p_F_y_abs_amp_layout)
    p_F_y_abs_amp_path = outputpath+r"\p_Amplitude_y {} to {}.png".format(a,b)
    Pngprint(p_F_y_abs_amp_fig,p_F_y_abs_amp_path)
    
    #peak_invSig(cut=just peak)
    p_i_sig_x=process.Amp2sig(p_F_x)
    p_i_sig_y=process.Amp2sig(p_F_y)
    p_i_sig_x*=2
    p_i_sig_y*=2

    p_i_sig_x_data = [F_Line(x=time,y=sig_x,color=Sigcolor_x),F_Line(x=time,y=p_i_sig_x,color=Ampcolor_y)]
    p_i_sig_x_layout = F_Layout(yran=SigRan_x,title="peak_invSignal of X {} to {}".format(a,b))
    p_i_sig_x_fig = F_Fig(p_i_sig_x_data,p_i_sig_x_layout)
    p_i_sig_x_path = outputpath+r"\peak_invSignal_X {} to {}.png".format(a,b)
    Pngprint(p_i_sig_x_fig,p_i_sig_x_path)

    p_i_sig_y_data = [F_Line(x=time,y=sig_y,color=Sigcolor_y),F_Line(x=time,y=p_i_sig_y,color=Ampcolor_x)]
    p_i_sig_y_layout = F_Layout(yran=SigRan_y,title="peak_invSignal of Y {} to {}".format(a,b))
    p_i_sig_y_fig = F_Fig(p_i_sig_y_data,p_i_sig_y_layout)
    p_i_sig_y_path = outputpath+r"\peak_invSignal_Y {} to {}.png".format(a,b)
    Pngprint(p_i_sig_y_fig,p_i_sig_y_path)
    
    #p_scatter
    p_i_sig_xy_data = [F_Line(x=p_i_sig_x,y=p_i_sig_y,color="black")]
    p_i_sig_xy_layout = F_Layout(xran=SigRan_x,yran=SigRan_y,title="peak_Scatter {} to {}".format(a,b))
    p_i_sig_xy_fig = F_Fig(p_i_sig_xy_data,p_i_sig_xy_layout)
    p_i_sig_xy_path = outputpath+r"\peak_scatter {} to {}.png".format(a,b)
    Pngprint(p_i_sig_xy_fig,p_i_sig_xy_path)

    #noise_rejection_HeatMap
    p_i_heat_data = F_2DHist(p_i_sig_x,p_i_sig_y)
    p_i_heat_layout = F_Layout(xran=SigRan_x,yran=SigRan_y,title="p_i_HeatMap {} to {}".format(a,b))
    p_i_heat_fig = F_Fig(p_i_heat_data,p_i_heat_layout)
    p_i_heat_path = outputpath+r"\p_i_HeatMap {} to {}.png".format(a,b)
    Pngprint(p_i_heat_fig,p_i_heat_path)
    
    return (peak_x,peak_y,peak_freq_x,peak_freq_y,sig_3d_data,sig_x_data,sig_y_data,amp_x_data,amp_y_data,(heat_data,heat_layout),inv_sig_x_data,inv_sig_y_data,nr_F_x_abs_amp_data,nr_F_y_abs_amp_data,nr_i_sig_x_data,nr_i_sig_y_data,nr_i_heat_data,p_F_x_abs_amp_data,p_F_y_abs_amp_data,p_i_sig_x_data,p_i_sig_y_data,p_i_sig_xy_data,p_i_heat_data,)

In [7]:
"""
x=[1,5,6]
y=[3,8,5]
x1=[5,6,7]
y1=[9,4,2]
x2=[3,2,11]
y2=[6,3,6]
data = F_Line(x,y)
data1 = F_Line(x1,y1)
data2 = F_Line(x2,y2)

framedata = [data,data1,data2]
frames = F_Frame(framedata)
layout = F_AnimLayout(frames)
fig = F_Fig(data = data1,layout=layout,frames=frames)
"""
#path = r"C:\Users\b215029\test.html"

#HtmlExport(fig,path)



'\nx=[1,5,6]\ny=[3,8,5]\nx1=[5,6,7]\ny1=[9,4,2]\nx2=[3,2,11]\ny2=[6,3,6]\ndata = F_Line(x,y)\ndata1 = F_Line(x1,y1)\ndata2 = F_Line(x2,y2)\n\nframedata = [data,data1,data2]\nframes = F_Frame(framedata)\nlayout = F_AnimLayout(frames)\nfig = F_Fig(data = data1,layout=layout,frames=frames)\n'

In [8]:
"""fig = go.Figure(
    data=[go.Scatter(x=[0, 1], y=[0, 1])],
    layout=go.Layout(
        xaxis=dict(range=[0, 5], autorange=False),
        yaxis=dict(range=[0, 5], autorange=False),
        title="Start Title",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None])])]
    ),
    frames=[go.Frame(data=[go.Scatter(x=[1, 2], y=[1, 2])]),
            go.Frame(data=[go.Scatter(x=[1, 4], y=[1, 4])]),
            go.Frame(data=[go.Scatter(x=[3, 4], y=[3, 4])],
                     layout=go.Layout(title_text="End Title"))]
)

fig.show()"""

'fig = go.Figure(\n    data=[go.Scatter(x=[0, 1], y=[0, 1])],\n    layout=go.Layout(\n        xaxis=dict(range=[0, 5], autorange=False),\n        yaxis=dict(range=[0, 5], autorange=False),\n        title="Start Title",\n        updatemenus=[dict(\n            type="buttons",\n            buttons=[dict(label="Play",\n                          method="animate",\n                          args=[None])])]\n    ),\n    frames=[go.Frame(data=[go.Scatter(x=[1, 2], y=[1, 2])]),\n            go.Frame(data=[go.Scatter(x=[1, 4], y=[1, 4])]),\n            go.Frame(data=[go.Scatter(x=[3, 4], y=[3, 4])],\n                     layout=go.Layout(title_text="End Title"))]\n)\n\nfig.show()'

In [9]:
def Test(inputpath,outputpath,Sigcolor_x,Ampcolor_x,Sigcolor_y,Ampcolor_y,SigRan_x,AmpRan_x,SigRan_y,AmpRan_y):
    #__init__
    if not os.path.exists(outputpath):
        os.mkdir(outputpath)
    print("{} is converting...".format(outputpath))

    peak_X=[]
    peak_Y=[]
    peak_index_X=[]
    peak_index_Y=[]
    peak_freq_X=[]
    peak_freq_Y=[]

    f_sig_3d =[]
    f_sig_x =[]
    f_sig_y =[]
    f_amp_x =[]
    f_amp_y =[]
    f_heat_data =[]
    f_heat_layout =[]
    f_inv_sig_x =[]
    f_inv_sig_y =[]
    f_nr_F_x_abs_amp =[]
    f_nr_F_y_abs_amp =[]
    f_nr_i_sig_x =[]
    f_nr_i_sig_y =[]
    f_nr_i_heat =[]
    f_p_F_x_abs_amp =[]
    f_p_F_y_abs_amp =[]
    f_p_i_sig_x =[]
    f_p_i_sig_y =[]
    f_p_i_sig_xy =[]
    f_p_i_heat =[]

    #samp=["0 to 10","10 to 20","20 to 30","30 to 40","40 to 50","50 to 60"]
    samp=[
        "00 to 10","","","","","","","","",
        "10 to 20","","","","","","","","","",
        "20 to 30","","","","","","","","","",
        "30 to 40","","","","","","","","","",
        "40 to 50","","","","","","","","",
        "50 to 60",
        ]
    rec=Rectangle(6001)
    N = np.linspace(0,60,num=6001)
    r_sig_x,r_sig_y,r_time = Csv(inputpath)

    #### Process ###
    time,sig_x = Liner(N,r_time,r_sig_x)
    time,sig_y = Liner(N,r_time,r_sig_y)

    time,sig_x = time*rec,sig_x*rec
    time,sig_y = time*rec,sig_y*rec

    #ALL
    T_Sig2amp(time,sig_x,sig_y,Sigcolor_x,Sigcolor_y,Ampcolor_x,Ampcolor_y,SigRan_x,SigRan_y,AmpRan_x,AmpRan_y,outputpath)
    #Sepalation
    sep_time = sep(time)
    sep_sig_x = sep(sig_x)
    sep_sig_y = sep(sig_y)

    #Roller
    roller_time = Roller(time)
    roller_sig_x = Roller(sig_x)
    roller_sig_y = Roller(sig_y)

    for i in range(len(roller_time)):
        p=T_Sig2amp(roller_time[i][0],roller_sig_x[i][0],roller_sig_y[i][0],Sigcolor_x,Sigcolor_y,Ampcolor_x,Ampcolor_y,SigRan_x,SigRan_y,AmpRan_x,AmpRan_y,outputpath)
        peak_X += [p[0]]
        peak_Y += [p[1]]
        peak_freq_X += [p[2]]
        peak_freq_Y += [p[3]]

        f_sig_3d += [p[4]]
        f_sig_x += [p[5]]
        f_sig_y += [p[6]]
        f_amp_x += [p[7]]
        f_amp_y += [p[8]]
        f_heat_data += [p[9][0]]
        f_heat_layout +=[p[9][1]]
        f_inv_sig_x += [p[10]]
        f_inv_sig_y += [p[11]]
        f_nr_F_x_abs_amp += [p[12]]
        f_nr_F_y_abs_amp += [p[13]]
        f_nr_i_sig_x += [p[14]]
        f_nr_i_sig_y += [p[15]]
        f_nr_i_heat += [p[16]]
        f_p_F_x_abs_amp += [p[17]]
        f_p_F_y_abs_amp += [p[18]]
        f_p_i_sig_x += [p[19]]
        f_p_i_sig_y += [p[20]]
        f_p_i_sig_xy += [p[21]]
        f_p_i_heat += [p[22]]
    
    f_heat_fig=AnimFig(f_heat_data)
    f_heat_path = outputpath+r"\f_heat.html"
    HtmlExport(f_heat_fig,f_heat_path)

    #Pngprint((F_Fig(sig_3d[0]))),outputpath+r"\peak_X.png")

    Pngprint((F_Fig(data=(F_Line(x=[i for i in range(len(peak_X))],y=peak_X,color="black")),layout=(F_Layout(yran=None,title="peak_X")))),outputpath+r"\peak_X.png")
    Pngprint((F_Fig(data=(F_Line(x=[i for i in range(len(peak_freq_X))],y=peak_freq_X,color="green")),layout=(F_Layout(yran=None,title="peak_freq_X")))),outputpath+r"\peak_freq_X.png")

    Pngprint((F_Fig(data=(F_Line(x=[i for i in range(len(peak_Y))],y=peak_Y,color="black")),layout=(F_Layout(yran=None,title="peak_Y")))),outputpath+r"\peak_Y.png")
    Pngprint((F_Fig(data=(F_Line(x=[i for i in range(len(peak_freq_Y))],y=peak_freq_Y,color="green")),layout=(F_Layout(yran=None,title="peak_freq_Y")))),outputpath+r"\peak_freq_Y.png")

    ########################################


    ########################################
    
    print(" Done.")
    print("")

def FourierTest(inputpath,outputpath):
    Test(
    inputpath = inputpath,
    outputpath = outputpath,

    ### Parameter ###
    Sigcolor_x="red",
    Ampcolor_x="orangered",
    Sigcolor_y="blue",
    Ampcolor_y="cyan",

    #range
    SigRan_x=[-30,30],
    AmpRan_x=[0,10],
    SigRan_y=[-30,30],
    AmpRan_y=[0,10]
    )

In [5]:
def Paramater():
    datasets=[
        #testdata.d_190714OCData(),
        testdata.d_honda(),
    ]
    for dataset in datasets:
        inputpath = dataset
        testdata.d_hukuma(),
        testdata.d_ikeda(),
        testdata.d_miyamoto(),
        testdata.d_morita(),
        testdata.d_nawate(),
        testdata.d_ohno(),
        testdata.d_orimoto(),
        testdata.d_printer(),
        """
        #printer
        r'.\data\printer\Printer200523200224.csv',
        r'.\data\printer\Printer200523200802.csv',
        #honda
        r'.\data\honda\BC00.csv',
        r'.\data\honda\BO00.csv',
        r'.\data\honda\TC00.csv',
        r'.\data\honda\TO00.csv',
        #nawate
        r'.\data\nawate\BC00.csv',
        r'.\data\nawate\BO00.csv',
        r'.\data\nawate\TC00.csv',
        r'.\data\nawate\TO00.csv',
        """
        base = [os.path.splitext(inputpath[_]) for _ in range(len(inputpath))]
        outputpath=[base[j][0] for j in range(len(base))]

        for i in range(len(outputpath)):
            print("No.{}".format(i+1))
            FourierTest(inputpath[i],outputpath[i])
    print("All Success.")

Paramater()

No.1
C:/Users/b215029/Desktop/CoG/data/Honda/BC00 is converting...


NameError: name 'figdata' is not defined

In [ ]:
def RecWin(low,high,end):
    recta = np.append((Rectangle(low)-1),Rectangle(high))
    recta = np.append(recta,(Rectangle(end)-1))
    return recta
RecWin(0,5,10)

honda = testdata.d_honda()
print(honda)

In [ ]:
"""""""""#FileChoice.App()

    #### Window 10sec ###
    def Sep10s(sep_time,sep_sig_x,sep_sig_y)
        for i in range(len(sn)):
            dt=0.001
            freq,amp_x,F_x = process.Sig2amp(time,sig_x,dt)
            freq,amp_y,F_y = process.Sig2amp(time,sig_y,dt)

            msx = max(fsx)
            msy = max(fsy)

            peak_X += [msx]
            peak_index_X += [np.argmax(fsx)]
            peak_Y += [msy]
            peak_index_Y += [np.argmax(fsy)]

            Pngprint((Figu(nsn,nsx,Sigcolor_X,yran=SigRan_X,title="Sample Signal_X {} to {}".format(i*10,(i+1)*10))),outputpath+r"\Sample Signal_X {} to {}.png".format(i*10,(i+1)*10))
            
            Pngprint((Figu(fsn,fsx,Ampcolor_X,yran=AmpRan_X,title="Sample Amplitude_X {} to {}".format(i*10,(i+1)*10))),outputpath+r"\Sample Amplitude_X .png".format(i*10,(i+1)*10))
            
            Pngprint((Figu(nsn,nsy,Sigcolor_Y,yran=SigRan_Y,title="Sample Signal_Y {} to {}".format(i*10,(i+1)*10))),outputpath+r"\Sample Signal_Y {} to {}.png".format(i*10,(i+1)*10))

            Pngprint((Figu(fsn,fsy,Ampcolor_Y,yran=AmpRan_Y,title="Sample Amplitude_Y {} to {}".format(i*10,(i+1)*10))),outputpath+r"\Sample Amplitude_Y {} to {}.png".format(i*10,(i+1)*10))
            
            Pngprint((Figu2(nsx,nsy,xran=SigRan_X,yran=SigRan_Y,title="Sample HeatMap {} to {}".format(i*10,(i+1)*10))),outputpath+r"\Sample all HeatMap {} to {}.png".format(i*10,(i+1)*10))


"""""""""